In [1]:
import os
import sys
import pandas as pd
import numpy as np
import logging
from flask import jsonify, make_response
from dotenv import load_dotenv
from datetime import date
from google.oauth2 import service_account

In [2]:
df_rooms = pd.read_csv('dataset/table_rooms.csv')
df_agreement = pd.read_csv('dataset/table_agreement.csv')
df_buildings = pd.read_csv('dataset/table_buildings.csv')

In [3]:
df_rooms.head()

,room_id,room_status,is_deleted,building_id,booked_remark,maintenance_status,booked_by_sales
0,10204,10,False,625,NaN,0,False
1,11351,9,False,669,NaN,0,False
2,11352,9,False,669,NaN,0,False
3,11353,9,False,669,NaN,0,False
4,11118,6,False,666,NaN,0,False


In [4]:
# join tables
df = pd.merge(df_buildings, df_rooms, how = 'left', left_on = 'id', right_on = 'building_id')
df = pd.merge(df, df_agreement, how = 'left', on = 'building_id')

# change data type
df['building_live_date'] = pd.to_datetime(df['building_live_date'])
df['soft_live_date'] = pd.to_datetime(df['soft_live_date'])

# Filtering data by building_live_date <= today
today = date.today()
df = df[df['building_live_date'] <= str(today)]

In [5]:
# create filter base on requirement
filtered_occupied = [
    (df['room_status'] != 10) & (df['room_status'] != 13) & df['live_date_confirm'] == True
    ]
occupied_values = [1]
df["occupied"] = np.select(filtered_occupied, occupied_values, default= 0)

In [6]:
# check each total occupied
df['occupied'].value_counts()

occupied
0    303
1    287
Name: count, dtype: int64

In [7]:
# get calculate occupied rooms
df_occupied_rooms_temp = df.groupby(["building_id","occupied"], as_index=False)["room_id"].count()
df_occupied_rooms_temp = df_occupied_rooms_temp.rename(columns = {"room_id" : "occupied_rooms"})
df = pd.merge(df, df_occupied_rooms_temp, how = 'inner', on = ['building_id','occupied'])

In [8]:
# get calculate total rooms
df_total_rooms_temp = df.groupby(["building_id"], as_index=False)["room_id"].count()
df_total_rooms_temp = df_total_rooms_temp.rename(columns = {"room_id" : "total_rooms_by_building"})
df = pd.merge(df, df_total_rooms_temp, how = 'inner', on = ['building_id'])

In [10]:
df = df[df["occupied"] == 1]
# calculate occupancy rooms
df["occupancy"] = df["occupied_rooms"] / df["total_rooms_by_building"]

/var/folders/13/j5z23k_s2xd0l5wj6l3zs8nw0000gn/T/ipykernel_55387/1919519654.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["occupancy"] = df["occupied_rooms"] / df["total_rooms_by_building"]


In [12]:
# set just column to ingest
df = df[["property_code","rukita_option","occupancy","building_live_date"]]
#rename base on requirement
df = df.rename(columns = {'building_live_date' : 'date'})
# delete duplicates
df = df.drop_duplicates()
# setup data type
df = df.astype({"property_code" : "string", "rukita_option" : "boolean", "occupancy" : "float", "date" : "datetime64[ns]"})
#order by
df = df.sort_values(by=["date", "property_code"])
# make sure data type is correct
print(df.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 461 to 85
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   property_code  27 non-null     string        
 1   rukita_option  27 non-null     boolean       
 2   occupancy      27 non-null     float64       
 3   date           27 non-null     datetime64[ns]
dtypes: boolean(1), datetime64[ns](1), float64(1), string(1)
memory usage: 918.0 bytes
None


In [13]:
df

,property_code,rukita_option,occupancy,date
461,RKT-JKT-CC-173,True,0.900000,2022-10-14
197,RKT-JKT-CC-172,True,0.200000,2022-10-21
219,RKT-SBY-CC-014,True,0.600000,2022-10-21
244,RKT-JKT-CC-174,True,0.920000,2022-10-28
273,RKT-JKT-CC-175,True,0.777778,2022-10-28
233,RKT-SBY-CC-015,True,0.600000,2022-10-28
99,RKT-SMN-CC-007,True,0.960000,2022-10-28
544,RKT-TGR-CC-070,True,0.500000,2022-10-28
136,RKT-DPK-CC-014,True,0.400000,2022-11-04
291,RKT-TGR-CC-071,True,0.750000,2022-11-04


In [15]:
import yaml
with open('schema/daily_occupancy.yml', 'r') as file:
    schema = yaml.safe_load(file)

print(schema)

[{'name': 'property_code', 'type': 'STRING'}, {'name': 'rukita_option', 'type': 'BOOLEAN'}, {'name': 'occupancy', 'type': 'FLOAT'}, {'name': 'date', 'type': 'DATETIME'}]


In [19]:
GBQ_PROJECT = "project-imam-dev"
GBQ_DATASET = "rukita"
GBQ_LOCATION = "asia-southeast1"

In [23]:
def insertToGbq(df:pd.DataFrame, project, dataset, table, schema, behaviour, location):
    credentials = service_account.Credentials.from_service_account_file('secret/sa-access.json',)
    destination = f"{dataset}.{table}"
    return df.to_gbq(project_id = project, credentials= credentials,destination_table = destination, table_schema = schema, if_exists = behaviour, location = location)

In [26]:
insertToGbq(df, GBQ_PROJECT, GBQ_DATASET, "daily_occupancy", schema, "replace", GBQ_LOCATION)

In [27]:
df.to_csv('output.csv', index=False)

Another Way

In [37]:
import yaml
from google.oauth2.service_account import Credentials
from google.cloud import bigquery

with open('schema/daily_occupancy.yml', 'r') as file:
    schema_load = yaml.safe_load(file)

def create_table_and_load_data(df, project, dataset_id, table_id, schema, parameter, location):
    # Load the service account credentials from the JSON file
    credentials = Credentials.from_service_account_file('secret/sa-access.json',)
    destination = f"{dataset_id}.{table_id}"

    # Create a BigQuery client using the service account credentials
    
    # client = bigquery.Client(credentials=credentials, project=project)

    # table_ref = client.dataset(dataset_id).table(table_id)
    # table = bigquery.Table(table_ref)

    
    return df.to_gbq(project_id = project, credentials= credentials,destination_table = destination, table_schema = schema, if_exists = parameter, location = location)
    # try:
    #     client.get_table(table)
    #     print(f"Table {table_id} already exists in BigQuery. Continuing to load data.")
    # except:
    #     print(f"Table {table_id} does not exist in BigQuery. Creating the table.")
    #     # Define the schema if necessary
    #     # schema = schema_load
    #     table.schema = schema
    #     table = client.create_table(table)
    #     print(f"Table {table_id} created in BigQuery.")

    # Load data from CSV file to BigQuery table
    #df.to_gbq(project_id = project, credentials= credentials,destination_table = f"{dataset_id}.{table_id}", table_schema = schema, if_exists = parameter, location = location)
    
    





In [38]:
create_table_and_load_data(df, GBQ_PROJECT, GBQ_DATASET, "occupancy_test", schema_load, "replace", GBQ_LOCATION)
print("Data loaded successfully.")

Data loaded successfully.
